In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-01-07


Found rundate 2024-01-06 10:14:26 < 2024-01-07 at commit 9ceb6138; returning commit ce58dbd


'2024-01-07 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-01-07 10:00:02, NJSP was reporting 4 YTD deaths
Current YTD Deaths (2025-01-07 10:00:02-05:00): 2
Previous year YTD Deaths (adjusted; 2024-01-07 10:00:02): 4.010958904109589
Projected 2025 total: 680.9515460416336
-50.1% change


FAUQStats cache miss: 2024, Sun Jan 07 10:00:02 EST 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,dt
ACCID,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,2024-01-03 12:14:00-05:00
12927,20,Union,2009,Linden City,27,27,1.0,0.0,0.0,1.0,0.0,2024-01-06 08:37:00-05:00
12928,15,Ocean,1505,Berkeley Twsp,9,9 (Business),1.0,0.0,0.0,1.0,0.0,2024-01-06 18:44:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Mercer,0,0,1,0,1
Middlesex,0,1,0,0,1


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

1.8% through the year, 98.2% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Mercer,driver,1,15,0
Middlesex,driver,0,29,0
Ocean,driver,0,28,0
Union,driver,0,18,0
Mercer,passenger,0,6,0
...,...,...,...,...
Passaic,crashes,0,32,0
Salem,crashes,0,14,0
Somerset,crashes,0,19,0


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,54,4,30,2,19
Bergen,38,2,15,5,19
Burlington,44,1,31,4,8
Camden,37,0,19,4,15
Cape May,9,0,8,0,1
Cumberland,27,0,19,6,7
Essex,48,0,18,10,27
Gloucester,28,0,18,8,6
Hudson,18,1,9,2,7


In [9]:
projected.to_csv(PROJECTED_CSV)